# Exercise 04 : Enrichment and transformations

## Imports

In [1]:
import pandas as pd
import numpy as np
import requests

## Read JSON file

In [2]:
df = pd.read_json('../data/auto.json', orient='records')

In [3]:
pd.options.display.float_format = '{:.2f}'.format
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus


## Enrich the dataframe

In [4]:
sample_200 = df.sample(n=200, random_state=21)
sample_200.head()

,CarNumber,Refund,Fines,Make,Model
445,M0299X197RUS,2,19200.00,Ford,Focus
22,83298C154RUS,2,8594.59,Ford,Focus
93,H957HY161RUS,1,2000.00,Ford,Focus
173,T941CC96RUS,1,2000.00,Ford,Focus
697,H966HY161RUS,1,500.00,Ford,Focus


In [5]:
sample_200['Fines'] = df['Fines'].sample(200).values
sample_200['Refund'] = df['Refund'].sample(200).values
sample_200.head()

,CarNumber,Refund,Fines,Make,Model
445,M0299X197RUS,1,6600.00,Ford,Focus
22,83298C154RUS,2,8594.59,Ford,Focus
93,H957HY161RUS,1,9200.00,Ford,Focus
173,T941CC96RUS,2,800.00,Ford,Focus
697,H966HY161RUS,2,13500.00,Ford,Focus


## Enrich concat_rows

In [6]:
concat_rows = pd.concat([df, sample_200])
concat_rows.reset_index(inplace=True, drop=True)
concat_rows

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
920,8182XX154RUS,1,12300.00,Ford,Focus
921,X796TH96RUS,1,3000.00,Ford,Focus
922,T011MY163RUS,2,400.00,Ford,Focus
923,T341CC96RUS,2,11400.00,Volkswagen,Passat


In [7]:
np.random.seed(21)
years = pd.Series(np.random.randint(1980, 2020, (len(concat_rows))), name="Year")
fines = pd.concat([concat_rows, years], axis=1)
fines.head()

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014


## Enrich with another dataframe

In [8]:
surnames = pd.read_json('../data/surname.json', orient='values')
surnames.columns = surnames.iloc[0].values
surnames.drop(surnames.index[0], axis='index', inplace=True)
surnames

,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
5,BAILEY,277845,72
...,...,...,...
96,WILLIAMS,1625252,3
97,WILSON,801882,14
98,WOOD,250715,84
99,WRIGHT,458980,35


In [9]:
owner_name = surnames['NAME'].sample(n=531, random_state=21, replace=True, ignore_index=True)
owner_name

0      RICHARDSON
1            ROSS
2          MORGAN
3          BAILEY
4           LOPEZ
          ...    
526      CAMPBELL
527          HALL
528         BAKER
529          DIAZ
530        MORGAN
Name: NAME, Length: 531, dtype: object

In [10]:
owner_car = concat_rows.drop_duplicates('CarNumber')['CarNumber']
owner_car = owner_car.reset_index(drop=True)
owner_car

0      Y163O8161RUS
1       E432XX77RUS
2       7184TT36RUS
3      X582HE161RUS
4      92918M178RUS
           ...     
526    O136HO197RUS
527    O22097197RUS
528    M0309X197RUS
529    O673E8197RUS
530    8610T8154RUS
Name: CarNumber, Length: 531, dtype: object

## Create `owners` dataframe

In [11]:
owners = pd.DataFrame(data=owner_car)
owners['Surname'] = owner_name
owners

,CarNumber,Surname
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


In [12]:
data1 = pd.Series(['SCHOOL21111RUS', 'SCHOOL21222RUS', 'SCHOOL21333RUS','SCHOOL21444RUS', 'SCHOOL21555RUS'], name='CarNumber')
data2 = pd.Series([np.random.choice(fines['Refund']) for i in range(5)], name= 'Refund')
data3 = pd.Series([np.random.choice(fines['Fines']) for i in range(5)], name= 'Fines')
data4 = pd.Series([np.random.choice(fines['Make']) for i in range(5)], name= 'Make')
data5 = pd.Series([np.random.choice(fines['Model']) for i in range(5)], name= 'Model')
data6 = pd.Series([np.random.choice(fines['Year']) for i in range(5)], name= 'Year')

df3 = pd.DataFrame([data1, data2, data3, data4, data5, data6])
fines = pd.concat([fines, df3.T])
fines.tail()

,CarNumber,Refund,Fines,Make,Model,Year
0,SCHOOL21111RUS,2,2400.00,Ford,Focus,2000
1,SCHOOL21222RUS,1,2000.00,Volkswagen,Focus,1996
2,SCHOOL21333RUS,2,1100.00,Ford,Focus,2013
3,SCHOOL21444RUS,1,1600.00,Ford,Focus,1992
4,SCHOOL21555RUS,2,1000.00,Ford,Focus,2005


In [14]:
owners_test = owners.drop(owners.tail(20).index)
data1 = pd.Series(['SCHOOL21666RUS', 'SCHOOL21777RUS', 'SCHOOL21888RUS'], name='CarNumber')
data2 = pd.Series([np.random.choice(owners['Surname']) for i in range(3)], name= 'Surname')
add3 = pd.DataFrame([data1, data2])
owners_test = pd.concat([owners_test, add3.T])
owners_test.tail

<bound method NDFrame.tail of           CarNumber     Surname
0      Y163O8161RUS  RICHARDSON
1       E432XX77RUS        ROSS
2       7184TT36RUS      MORGAN
3      X582HE161RUS      BAILEY
4      92918M178RUS       LOPEZ
..              ...         ...
509    O50197197RUS      WRIGHT
510    7608EE777RUS        HILL
0    SCHOOL21666RUS      WATSON
1    SCHOOL21777RUS       JONES
2    SCHOOL21888RUS     SANCHEZ

[514 rows x 2 columns]>

In [15]:
test1 = pd.merge(left=fines, right=owners_test, on='CarNumber', how='inner')
test1

,CarNumber,Refund,Fines,Make,Model,Year,Surname
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,Y163O8161RUS,2,1600.00,Ford,Focus,1980,RICHARDSON
2,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
3,E432XX77RUS,2,13000.00,Toyota,Camry,2018,ROSS
4,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
...,...,...,...,...,...,...,...
894,E41977152RUS,2,2400.00,Ford,Focus,1989,BAKER
895,9464EX178RUS,2,2100.00,Ford,Focus,1988,MARTIN
896,O50197197RUS,2,7800.00,Ford,Focus,1992,WRIGHT
897,7608EE777RUS,1,4000.00,Skoda,Octavia,2000,HILL


In [16]:
test2 = pd.merge(fines, owners_test, on='CarNumber', how='outer')
test2

,CarNumber,Refund,Fines,Make,Model,Year,Surname
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,Y163O8161RUS,2,1600.00,Ford,Focus,1980,RICHARDSON
2,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
3,E432XX77RUS,2,13000.00,Toyota,Camry,2018,ROSS
4,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
...,...,...,...,...,...,...,...
928,SCHOOL21444RUS,1,1600.00,Ford,Focus,1992,NaN
929,SCHOOL21555RUS,2,1000.00,Ford,Focus,2005,NaN
930,SCHOOL21666RUS,NaN,NaN,NaN,NaN,NaN,WATSON
931,SCHOOL21777RUS,NaN,NaN,NaN,NaN,NaN,JONES


In [17]:
test3 = pd.merge(fines, owners_test, on='CarNumber', how='left')
test3

,CarNumber,Refund,Fines,Make,Model,Year,Surname
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
925,SCHOOL21111RUS,2,2400.00,Ford,Focus,2000,NaN
926,SCHOOL21222RUS,1,2000.00,Volkswagen,Focus,1996,NaN
927,SCHOOL21333RUS,2,1100.00,Ford,Focus,2013,NaN
928,SCHOOL21444RUS,1,1600.00,Ford,Focus,1992,NaN


In [18]:
test4 = pd.merge(fines, owners_test, on='CarNumber', how='right')
test4

,CarNumber,Refund,Fines,Make,Model,Year,Surname
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,Y163O8161RUS,2,1600.00,Ford,Focus,1980,RICHARDSON
2,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
3,E432XX77RUS,2,13000.00,Toyota,Camry,2018,ROSS
4,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
...,...,...,...,...,...,...,...
897,7608EE777RUS,1,4000.00,Skoda,Octavia,2000,HILL
898,7608EE777RUS,1,7900.00,Skoda,Octavia,1991,HILL
899,SCHOOL21666RUS,NaN,NaN,NaN,NaN,NaN,WATSON
900,SCHOOL21777RUS,NaN,NaN,NaN,NaN,NaN,JONES


## Create a pivot table from the `fines` dataframe in specified format

In [19]:
pd.pivot_table(fines, columns='Year', values='Fines', index=['Make', 'Model'], aggfunc={'Fines': np.sum})

Year                   1980      1981      1982      1983      1984      1985  \
Make       Model                                                                
Ford       Focus   64089.17 411689.17 220878.35 105900.00 104694.59 126678.35   
           Mondeo       NaN       NaN       NaN       NaN       NaN       NaN   
Skoda      Octavia 54500.00       NaN  19900.00  11594.59       NaN  10294.59   
Toyota     Camry   12000.00   8594.59       NaN   7200.00       NaN       NaN   
           Corolla      NaN       NaN   2000.00       NaN       NaN       NaN   
Volkswagen Focus        NaN       NaN       NaN       NaN       NaN       NaN   
           Golf    30900.00       NaN       NaN   8594.59    300.00  24000.00   
           Jetta        NaN       NaN       NaN       NaN       NaN       NaN   
           Passat       NaN   1600.00       NaN   3200.00  10000.00   5000.00   
           Touareg      NaN       NaN       NaN       NaN       NaN   5800.00   

Year                   1986     1987     1988      1989  ...      2010  \
Make       Model                                         ...             
Ford       Focus   96089.17 71300.00 94994.59 206494.59  ... 144783.76   
           Mondeo       NaN      NaN      NaN   8600.00  ...       NaN   
Skoda      Octavia   600.00  5200.00 10600.00  91400.00  ...   3100.00   
Toyota     Camry        NaN      NaN      NaN  22400.00  ...       NaN   
           Corolla      NaN 24200.00      NaN   4000.00  ...  24000.00   
Volkswagen Focus        NaN      NaN      NaN       NaN  ...       NaN   
           Golf         NaN 14900.00      NaN   5800.00  ...       NaN   
           Jetta        NaN      NaN      NaN       NaN  ...       NaN   
           Passat  15000.00 12300.00      NaN       NaN  ...   2800.00   
           Touareg      NaN      NaN      NaN       NaN  ...   6300.00   

Year                    2011     2012      2013      2014      2015     2016  \
Make       Model                                                               
Ford       Focus   124089.17 93300.00 162794.59 379189.17 218694.59 80994.59   
           Mondeo        NaN 34400.00       NaN       NaN       NaN 46200.00   
Skoda      Octavia    500.00   500.00  12594.59    300.00  46394.59   300.00   
Toyota     Camry         NaN  8594.59       NaN   3900.00       NaN      NaN   
           Corolla   8594.59  2500.00       NaN       NaN       NaN  8000.00   
Volkswagen Focus         NaN      NaN       NaN       NaN       NaN      NaN   
           Golf       300.00      NaN   9500.00       NaN   2300.00      NaN   
           Jetta         NaN      NaN       NaN       NaN       NaN      NaN   
           Passat        NaN      NaN       NaN       NaN    600.00  2100.00   
           Touareg       NaN      NaN       NaN   1300.00    500.00      NaN   

Year                    2017      2018     2019  
Make       Model                                 
Ford       Focus   276789.17 280489.17 63000.00  
           Mondeo        NaN       NaN      NaN  
Skoda      Octavia    800.00 156200.00  9500.00  
Toyota     Camry         NaN  15000.00 18100.00  
           Corolla   9600.00  18800.00      NaN  
Volkswagen Focus         NaN       NaN      NaN  
           Golf          NaN   7000.00      NaN  
           Jetta         NaN       NaN      NaN  
           Passat        NaN       NaN      NaN  
           Touareg       NaN       NaN      NaN  

[10 rows x 40 columns]

## Save both the `fines` and `owners` dataframes to CSV files without an index

In [20]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)